## 잠재요인 협업 필터링

In [1]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095457 sha256=d1f6a7a238eaab7e7008db692197211811b9e45f48164c45040d6c3964b3e3ea
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
# 데이터 파일 업로드
from google.colab import files
up = files.upload()

Saving movies.csv to movies.csv
Saving ratings.csv to ratings.csv
Saving ratings_noh.csv to ratings_noh.csv


In [3]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
# 전체 데이터의 갯수, 사용자수, 영화수
ratings.shape[0], ratings.userId.nunique(), ratings.movieId.nunique()

(100836, 610, 9724)

#### 모델 생성 및 학습

In [5]:
from surprise import SVD, Reader
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data_folds = DatasetAutoFolds('ratings_noh.csv', reader=reader)

In [6]:
# 전체 데이터를 학습 데이터로 사용
trainset = data_folds.build_full_trainset()

In [8]:
# 모델 생성 및 학습
model = SVD(n_epochs=20, n_factors=50, random_state=2023, verbose=1)
model.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


- 테스트/확인: 사용자 id:9, 영화 id: 42 (Dead Presidents (1995))

In [9]:
# 영화 제목 정보
mdf = pd.read_csv('movies.csv')
mdf[mdf.movieId == 42]

,movieId,title,genres
38,42,Dead Presidents (1995),Action|Crime|Drama


In [12]:
# userId=9 인 사용자가 movieId=42 인 영화를 보았는지 확인
movieIds = ratings[ratings.userId == 9]['movieId']
movieIds[movieIds == 42].count()

0

In [14]:
try:
    movieIds[42]
except:
    print('42번 영화를 보지 않았습니다.')

42번 영화를 보지 않았습니다.


In [15]:
pred = model.predict(uid='9', iid='42', verbose=True)

user: 9          item: 42         r_ui = None   est = 2.96   {'was_impossible': False}


In [16]:
pred

Prediction(uid='9', iid='42', r_ui=None, est=2.958615033183178, details={'was_impossible': False})

- 사용자 id 9번이 보지 않은 영화중에서 예상점수가 가장 높은 Top 10

In [17]:
total_movies = mdf.movieId.tolist()
seen_movies = ratings[ratings.userId == 9]['movieId'].tolist()
unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
len(seen_movies), len(unseen_movies)

(46, 9696)

In [18]:
predictions = []
for mid in unseen_movies:
    pred = model.predict(uid='9', iid=str(mid))
    predictions.append(pred)
predictions[:5]

[Prediction(uid='9', iid='1', r_ui=None, est=3.4776519829512216, details={'was_impossible': False}),
 Prediction(uid='9', iid='2', r_ui=None, est=3.2112541390330303, details={'was_impossible': False}),
 Prediction(uid='9', iid='3', r_ui=None, est=2.8472928958780246, details={'was_impossible': False}),
 Prediction(uid='9', iid='4', r_ui=None, est=2.729956004026319, details={'was_impossible': False}),
 Prediction(uid='9', iid='5', r_ui=None, est=2.610270730107314, details={'was_impossible': False})]

In [19]:
def sortkey_est(pred):
    return pred.est

In [20]:
predictions.sort(key=sortkey_est, reverse=True)
predictions[:5]

[Prediction(uid='9', iid='2959', r_ui=None, est=4.103662366606615, details={'was_impossible': False}),
 Prediction(uid='9', iid='7361', r_ui=None, est=4.095549306544156, details={'was_impossible': False}),
 Prediction(uid='9', iid='858', r_ui=None, est=4.0944254123065456, details={'was_impossible': False}),
 Prediction(uid='9', iid='260', r_ui=None, est=4.093231006166612, details={'was_impossible': False}),
 Prediction(uid='9', iid='246', r_ui=None, est=4.062247437031265, details={'was_impossible': False})]

In [21]:
top_movie_ids = [int(pred.iid) for pred in predictions[:10]]
top_movie_ratings = [pred.est for pred in predictions[:10]]
top_movie_titles = []
for mid in top_movie_ids:
    title = mdf[mdf.movieId == mid]['title'].values[0]
    top_movie_titles.append(title)

In [22]:
df = pd.DataFrame({
    'movieId': top_movie_ids, 'title': top_movie_titles, 'rating': top_movie_ratings
})
df

,movieId,title,rating
0,2959,Fight Club (1999),4.103662
1,7361,Eternal Sunshine of the Spotless Mind (2004),4.095549
2,858,"Godfather, The (1972)",4.094425
3,260,Star Wars: Episode IV - A New Hope (1977),4.093231
4,246,Hoop Dreams (1994),4.062247
5,1196,Star Wars: Episode V - The Empire Strikes Back...,4.059472
6,7153,"Lord of the Rings: The Return of the King, The...",4.058896
7,1208,Apocalypse Now (1979),4.051314
8,6711,Lost in Translation (2003),4.044588
9,318,"Shawshank Redemption, The (1994)",4.039564


- 사용자 id를 제공하면 추천 영화를 반환하는 함수

In [26]:
def sortkey_est(pred):
    return pred.est

def get_top10_movies(uid):                  # uid는 정수
    total_movies = mdf.movieId.tolist()
    seen_movies = ratings[ratings.userId == uid]['movieId'].tolist()
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    predictions = []
    for mid in unseen_movies:
        pred = model.predict(uid=str(uid), iid=str(mid))
        predictions.append(pred)
    predictions.sort(key=sortkey_est, reverse=True)
    top_movie_ids = [int(pred.iid) for pred in predictions[:10]]
    top_movie_ratings = [pred.est for pred in predictions[:10]]
    top_movie_titles = [mdf[mdf.movieId == mid]['title'].values[0] for mid in top_movie_ids]
    df = pd.DataFrame({'movieId': top_movie_ids, 'title': top_movie_titles, 'rating': top_movie_ratings})
    return df

In [27]:
get_top10_movies(100)

,movieId,title,rating
0,904,Rear Window (1954),4.694956
1,1258,"Shining, The (1980)",4.661720
2,898,"Philadelphia Story, The (1940)",4.661173
3,475,In the Name of the Father (1993),4.647624
4,1945,On the Waterfront (1954),4.634245
5,3275,"Boondock Saints, The (2000)",4.629319
6,5690,Grave of the Fireflies (Hotaru no haka) (1988),4.622983
7,58559,"Dark Knight, The (2008)",4.615440
8,2324,Life Is Beautiful (La Vita è bella) (1997),4.613965
9,1204,Lawrence of Arabia (1962),4.605477
